<a href="https://colab.research.google.com/github/TrongNgoVan/Nam-4_Ky-1_NhapMonKhoaHocDuLieu-PTIT/blob/master/GNN_LSTM_Neo4J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install py2neo
!pip install stellargraph
!pip install networkx
# dùng py2neo để liên kết với Neo4J,
# dùng stellargraph để tạo node embedding đồ thị
!pip install neo4j py2neo


ERROR: Ignored the following versions that require a different python version: 0.10.0 Requires-Python >=3.6.0, <3.8.0; 0.11.0 Requires-Python >=3.6.0, <3.8.0; 0.11.1 Requires-Python >=3.6.0, <3.8.0; 0.4.0 Requires-Python >=3.6,<3.7; 0.4.0b0 Requires-Python >=3.6,<3.7; 0.4.1 Requires-Python >=3.5.0, <3.7.0; 0.5.0 Requires-Python >=3.5.0, <3.7.0; 0.6.0 Requires-Python >=3.5.0, <3.7.0; 0.6.1 Requires-Python >=3.5.0, <3.7.0; 0.7.0 Requires-Python >=3.5.0, <3.8.0; 0.7.1 Requires-Python >=3.5.0, <3.8.0; 0.7.2 Requires-Python >=3.5.0, <3.8.0; 0.7.3 Requires-Python >=3.5.0, <3.8.0; 0.8.0 Requires-Python >=3.5.0, <3.8.0; 0.8.1 Requires-Python >=3.5.0, <3.8.0; 0.8.2 Requires-Python >=3.5.0, <3.8.0; 0.8.3 Requires-Python >=3.5.0, <3.8.0; 0.8.4 Requires-Python >=3.5.0, <3.8.0; 0.9.0 Requires-Python >=3.6.0, <3.8.0; 1.0.0 Requires-Python >=3.6.0, <3.8.0; 1.0.0rc1 Requires-Python >=3.6.0, <3.8.0; 1.1.0 Requires-Python >=3.6.0, <3.9.0; 1.2.0 Requires-Python >=3.6.0, <3.9.0; 1.2.1 Requires-Python >=3.

In [4]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

# Thông tin kết nối
URI = "neo4j+s://eb4296fe.databases.neo4j.io"
AUTH = ("neo4j", "trong1308")

# Kết nối và kiểm tra
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Kết nối thành công tới Neo4j!")


def execute_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]


query = "MATCH (n) RETURN n LIMIT 1000"
results = execute_query(query)
for record in results:
    print(record)

graph = GraphDatabase("neo4j+s://eb4296fe.databases.neo4j.io", auth=("neo4j", "trong1308"))
data = graph.run(query).data()

# Chuyển đổi dữ liệu thành DataFrame
edges_df = pd.DataFrame(data)
edges_df.head()
edges_df = pd.DataFrame(data)
edges_df.head()

#  Tạo ma trận kề
# Lấy danh sách các nút
nodes = list(set(edges_df['source']).union(set(edges_df['target'])))
node_index = {node: idx for idx, node in enumerate(nodes)}

# Khởi tạo ma trận kề
adj_matrix = np.zeros((len(nodes), len(nodes)))

# Điền vào ma trận kề
for _, row in edges_df.iterrows():
    source = row['source']
    target = row['target']
    adj_matrix[node_index[source], node_index[target]] = 1

adj_matrix[:5, :5]  # Xem trước một phần ma trận kề

query_features = """
MATCH (s:Store)
RETURN s.id AS node_id, s.affiliateRevenue AS affiliate_revenue,
       s.storeRevenue AS store_revenue, s.avgPrice AS avg_price,
       s.salesVolume AS sales_volume
"""
node_features_data = graph.run(query_features).data()
features_df = pd.DataFrame(node_features_data)

# Tạo ma trận đặc trưng với số đặc trưng là 4
features_matrix = np.zeros((len(nodes), 4))

for _, row in features_df.iterrows():
    node_id = row['node_id']
    idx = node_index[node_id]
    features_matrix[idx, 0] = row['affiliate_revenue']
    features_matrix[idx, 1] = row['store_revenue']
    features_matrix[idx, 2] = row['avg_price']
    features_matrix[idx, 3] = row['sales_volume']

features_matrix[:5]  # Xem trước ma trận đặc trưng

AuthError: {code: Neo.ClientError.Security.Unauthorized} {message: The client is unauthorized due to authentication failure.}